Item 18 Know How to Construct Key-Dependent Default Values with __missing__ 

Things to Remember
- The setdefault method of dict is a bad fit when creating the default value has high computational cost or may raise exceptions
- The function passed to defaultdict must not require any arguments, which makes it impossible to have the default value depend on the key being accessed
- You can define your own dict subclass with a __missing__ method in order to construct default values that must know which key was being accessed

In [ ]:
pictures = {}
path = 'cat.png'

In [ ]:
# - map picture pathnames to open file handlers
# - you will attempt to open the file only if the
#   path does not exit in the dict yet
# - worst case: one access and one assignment

if (handle := pictures.get(path)) is None: # access key
    try:
        # different key (path) will have different value (handler)
        handle = open(path, 'r+b') # file handler
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        # assignment
        pictures[path] = handle # add the handler to the dict
handle.seek(0)
image_data = handle.read()
print(pictures)

In [ ]:
# let see if you can use the setdefault method approach
try:
    handle = pictures.setdefault(path, open(path, 'r+b'))
except OSError:
    print(f'Failed to open path {path}')
    raise
else:
    handle.seek(0)
    image_data = handle.read() 

print(pictures)


Two issues with the setdefault method approach
- whenever you access the key the open method will always be called even if the key already exists in the dict
- if both setdefault and open raise exceptions at the same time you might not be able to differentiate them 


In [ ]:
# how about using defaultdict
from collections import defaultdict

def open_picture(profile_path):
    try:
        return open(path, 'r+b')
    except OSError:
        print(f'Failed to open path {profile_path}')
        raise
# - the function you pass to the constructor can't have
#   arguments 
pictures = defaultdict(open_picture) # error:  missing 1 required positional argument  
handle = pictures[path]
handle.seek(0)
image_data = handle.read()



In [13]:
# subclass the dict type and implement the __missing__ special method
class Pictures(dict):
    def __missing__(self, key):
        value = open_picture(key)
        self[key] = value
        return value
pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

- the pictures\[path\] dictionary access finds that the path key isn't present in the dictionary, the __missing__ method is called
- the __missing__ method must create the value for the key, insert it into the dictionary, and return it to the caller
- Subsequent accesses of the same path will not call __missing__ since the corresponding item is already present